In [ ]:
import pandas as pd

In [ ]:
bookings_df1 = """
503200025365
503200025390
503200025390
503200025403
503200025705
503200025705
503200025845""".split()

bookings_df2 = """
503200025365
503200025390.0
503200025390
503200025403
503200025403
503200025403
503200025705
503200025705
""".split()

containers_df2 ="""
DFSU6896599
EISU9363352
TGBU6778406
EGHU1003711
EGHU1019723
RTYH1256789
QWER1234567
TYUI2536890
""".split()

nwt = """
9990
8344
14692
14040
11520
12850
12720
""".split()
nwt2 = """
13.79
12.22
18.39
17.67
15.15
16.47
""".split()


In [ ]:
df1 = pd.DataFrame({'BOOKING': bookings_df1})
df2 = pd.DataFrame({'BOOKING': bookings_df2})

df_concat = pd.concat([df1, df2])
booking_array = df_concat['BOOKING'].unique()

START

In [1]:
import xlwings as xw
import pandas as pd

def egl_si():
    file_path = r"docs\si\RDX1 005S SEGOT.xlsm"
    df = pd.read_excel(file_path, sheet_name='Output', header=8, usecols=("A,C,E,I:K"))
    df = df.rename(columns={
        '  CONTAINER NO. '      : 'CONTAINER',
        ' PACKAGE'              : 'PACKAGES',
        '    GWT '              : 'NET WEIGHT',
        '    B/L NO.  '         : 'BOOKING NUMBER',
        ' MRN No / Q-status '   : 'MRN',
        ' PARTNER'              : 'MLO'
        })
    df = df.drop(index=0)
    df = df.dropna(axis='index', how='all')
    df[['CONTAINER', 'ISO TYPE']] = df['CONTAINER'].str.split(expand=True)
    df['BOOKING NUMBER'] = df['BOOKING NUMBER'].str.replace(' ', '')
    df.loc[:,'BOOKING NUMBER'] = df['BOOKING NUMBER'].astype(str).replace(r'\.0*', '')
    df = df.reset_index(drop=True)
    df = df.sort_values('BOOKING NUMBER')
    return df

def load_df():
    wb = xw.Book(r"docs\0114_Bokningsblad_ANINA_23004_SEGOT-ursprunglig.xlsb")
    sheet = wb.sheets('INFO')
    data_table = sheet.range('A4').expand()
    df = sheet.range(data_table).options(pd.DataFrame, index=False, header=True).value
    df = df.loc[(df.MLO == 'EVER') & (df['LOAD STATUS'] != 'MT'), :]
    df = df[['BOOKING NUMBER', 'MLO', 'CONTAINER', 'ISO TYPE', 'NET WEIGHT', 'LOAD STATUS', 'VGM', 'OOG', 'REMARK', 'IMDG', 'UNNR', 'MRN', 'TEMP', 'PACKAGES']]
    df = pd.DataFrame(df)
    df['BOOKING NUMBER'] = df['BOOKING NUMBER'].astype(str)
    df['BOOKING NUMBER'] = df['BOOKING NUMBER'].str.replace('.0', '', regex=False)
    df.insert(0, 'COMMENT', 'Bokningsblad')
    df.loc[:,'BOOKING NUMBER'] = df['BOOKING NUMBER'].astype(str).replace('.0', '', regex=False)
    df['MLO'] = None
    df['CONTAINER'] = None
    df['ISO TYPE'] = None
    df['NET WEIGHT'] = None
    df['COMMENT'] = "Bokningsblad"
    df = df.sort_values('BOOKING NUMBER').reset_index()
    
    return df

In [2]:
df1 = load_df()
df2 = egl_si()

#df1 = df1.rename(columns={'BOOKING NUMBER': 'BOOKING'})
#df1['BOOKING'] = df1['BOOKING'].astype(str)
#df1['BOOKING'] = df1['BOOKING'].str.replace('.0', '', regex=False)

df_concat = pd.concat([df1, df2])
booking_array = df_concat['BOOKING NUMBER'].unique()

#df1['COMMENT'] = "Bokningsblad"

Output


In [3]:
def booking_count(df1: pd.DataFrame, df2: pd.DataFrame, booking: str) -> int:
    df1_count, df2_count = 0, 0

    if booking in df1['BOOKING NUMBER'].values:
        df1_count = df1['BOOKING NUMBER'].value_counts()[booking]
    if booking in df2['BOOKING NUMBER'].values:
        df2_count = df2['BOOKING NUMBER'].value_counts()[booking]
        
    return max(df1_count, df2_count)

In [4]:

string = ""
string += ''.join(f'{booking},' * booking_count(df1, df2, booking) for booking in booking_array)

booking_list = ""
booking_list = string.split(',')

booking_list.pop()

df4 = pd.DataFrame(columns=df1.columns)
df4['BOOKING NUMBER'] = booking_list


In [ ]:
len(booking_list)

In [ ]:
booking_list

In [ ]:
df4

In [ ]:
booking_array

In [ ]:
df2

In [ ]:
df4

In [ ]:
df1

In [ ]:
df4['BOOKING NUMBER'] == df1['BOOKING NUMBER'][3]

In [ ]:
df2.columns.drop('BOOKING NUMBER')

In [13]:
counter_df1 = 0
counter_df2 = 0
df1_cols = df1.columns
df2_cols = df2.columns.drop('BOOKING NUMBER')

for num, booking in enumerate(df4['BOOKING NUMBER']):
    print(f"{num} {booking} df1: {df1['BOOKING NUMBER'][counter_df1]}/ {counter_df1} df2: {df2['BOOKING NUMBER'][counter_df2]}/ {counter_df2}")
    if not counter_df1 == len(df1):
        if booking == df1['BOOKING NUMBER'][counter_df1]:
            df4.loc[num, 'COMMENT'] = df1['COMMENT'][counter_df1]
            counter_df1 += 1

    if not counter_df2 == len(df2):
        if booking == df2['BOOKING NUMBER'][counter_df2]:
            df4.loc[num, df2_cols] = df2.loc[counter_df2, df2_cols]
            counter_df2 += 1


df4.to_excel('test.xlsx')
    





0 503200025365 df1: 503200025365/ 0 df2: 503200025365/ 0
1 503200025390 df1: 503200025390/ 1 df2: 503200025390/ 1
2 503200025390 df1: 503200025390/ 2 df2: 503200025390/ 2
3 5032000254009 df1: 5032000254009/ 3 df2: 503200025403/ 3
4 5032000254012 df1: 5032000254012/ 4 df2: 503200025403/ 3
5 503200025403 df1: 503200025403/ 5 df2: 503200025403/ 3
6 503200025403 df1: 503200025403/ 6 df2: 503200025403/ 4
7 503200025403 df1: 503200025403/ 7 df2: 503200025403/ 5
8 503200025403 df1: 503200025403/ 8 df2: 503200025403/ 6
9 503200025403 df1: 503200025403/ 9 df2: 503200025403/ 7
10 503200025403 df1: 503200025403/ 10 df2: 503200025403/ 8
11 503200025403 df1: 503200025403/ 11 df2: 503200025403/ 9
12 503200025403 df1: 503200025705/ 12 df2: 503200025403/ 10
13 503200025403 df1: 503200025705/ 12 df2: 503200025403/ 11
14 503200025403 df1: 503200025705/ 12 df2: 503200025403/ 12
15 503200025705 df1: 503200025705/ 12 df2: 503200025705/ 13
16 503200025705 df1: 503200025705/ 13 df2: 503200025705/ 14
17 50320

In [15]:
pd.DataFrame({'BOOKING': {'503300000674': ['EMCU8909929', 'TLLU4025385', 'TGBU4276667'], '503300000186': ['TGBU6778406', 'EGHU1003711']}})

,BOOKING
503300000186,"[TGBU6778406, EGHU1003711]"
503300000674,"[EMCU8909929, TLLU4025385, TGBU4276667]"
